# Prerequisites
For this web server to work:
1. Fill in the ngrok auth key in the below cell
2. Upload `twitterWordCloud.html` from the GitHub repo to a folder called `templates`
3. Upload a file called `bearer_token.txt` to the root directory with the Twitter API Bearer Token
4. Run the below cell and restart the runtime (to get the latest version of Tweepy that supports Twitter API v2)

In [1]:
!pip install --upgrade tweepy textblob matplotlib pandas
!python -m textblob.download_corpora
!pip install flask-ngrok
!pip install flask-bootstrap
# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
# extract the downloaded file using the following command 
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
# fill in the ngrok auth token here
!./ngrok authtoken 2GAklkQJRqujnvZ3lcQh3iGTh75_6HS7NUQqMtEfxnHmx2XPR

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 636 kB 37.5 MB/s 
     |████████████████████████████████| 11.2 MB 46.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 960 kB 47.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
  Attempting uninstall: textblob
    Found existing installation: textblob 0.15.3
    Uninstalling textblob-0.15.3:
      Successfully uninstalled textblob-0.15.3
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Success

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 456 kB 5.2 MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3

In [1]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template
import tweepy
import string
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import os
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

app = Flask(__name__)
run_with_ngrok(app)

stop_words = set(stopwords.words("english"))
twitter_bearer_token = open("bearer_token.txt", "r").read()
client = tweepy.Client(twitter_bearer_token)

@app.route('/')
def home():
  return render_template("twitterWordCloud.html")


@app.route('/getWordMap')
def getWordMap():
    sentiments = {}
    words = {}

    def search_twitter(search_query):
        paginator = tweepy.Paginator(client.search_recent_tweets, 
                                     query=search_query, 
                                     max_results=100, 
                                     limit=5)
        for tweet in paginator.flatten(limit=500):
            build_dictionary(tweet)
            get_sentiment(tweet)


    def sanitise_word(word):
        w = word.lower()
        w = w.translate(str.maketrans('', '', string.punctuation))
        if w not in stop_words and len(w) > 1 and w != "&amp;":
            return w
        return ""
        
    def build_dictionary(tweet):
        tweet_words = [sanitise_word(t) for t in tweet.text.split(" ")]
        for word in tweet_words:
            if word not in words and word != "":
                words[word] = 1
            elif word != "":
                words[word] += 1

    def round_nearest(x, a):
        return round(round(x / a) * a, 1)
    
    def get_sentiment(tweet):
        sentiment = TextBlob(tweet.text).sentiment.polarity
        nearest_category = round_nearest(sentiment, 0.2)
        if nearest_category not in sentiments:
            sentiments[nearest_category] = 1
        else:
            sentiments[nearest_category] += 1
    
    search_term = request.args.get("word")
  
    if search_term is None:
        return "[]" # Return empty array to the client so the word map generates nothing
    
    search_query = f"({search_term}) -is:retweet lang:en"
    search_twitter(search_query)
    # Sort dictionary of words by number of mentions, where the number of mentions is greater than 10
    sorted_dict = [{"x": t, "value": words[t]} for t in dict(sorted(words.items(), key=lambda item: item[1], reverse=True)) if words[t] > 10]

    response = jsonify({"words": sorted_dict, "sentiments": sentiments})
    response.headers.add("Access-Control-Allow-Origin", "*") # Enable CORS so the front-end canc be hosted elsewhere and still can access
    return response
if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e9a5-35-237-235-209.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Oct/2022 09:18:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Oct/2022 09:18:55] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Oct/2022 09:19:00] "GET /getWordMap?word=ada%20lovelace HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Oct/2022 09:19:00] "GET / HTTP/1.1" 200 -
